In [23]:
# Load required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Load the data
df = pd.read_csv('train_info.csv')

# Keep only the columns we need: filename and style
df = df[['filename', 'style']]

# Drop rows with blank styles
df = df.dropna()

# Drop rows with styles that have less than 500 images
df = df[df['style'].map(df['style'].value_counts()) > 500]

# Show counts for each style
df['style'].value_counts()

In [ ]:
# Split the data into train and test sets with a 90/10 split and stratify by style
train, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['style'])

# Split the train set into train and validation sets with an 89/11 split and stratify by style
# This will result in an overall split of 80.1/9.9/10 for train/validation/test
train, val = train_test_split(train, test_size=0.11, random_state=42, stratify=train['style'])